# Solve cases
In this notebook, a case study is selected and solved with the A* implementation. 

## Import classes

In [1]:
from ipynb.fs.full.classes import *
from ipynb.fs.full.a_star import *
import numpy as np

In [2]:
# import sys
# !{sys.executable} -m pip install networkx

# Choose case setting
Choose only one of the files in the below code cell! They have different graphs, which shouldn't exist in memory at the same time, as that could lead to warnings (graphs for variables that have the same names will overwrite). Restart the kernel before selecting a different file.

In [3]:
#from ipynb.fs.full.case_abstract_small import *
#from ipynb.fs.full.case_abstract_medium import *
#from ipynb.fs.full.case_abstract_large import *
from ipynb.fs.full.case_mobility import *

## Overview of test cases
Tests in `case_mobility`:
- `test_mobility`: as discussed in the paper

Tests in `case_abstract_small`:
- `test_1`: easy case, possible in 4 steps
- `test_2`: not possible

Tests in `case_abstract_medium`: 
- `test_1`: possible in 11 steps or quicker, depending on order
- `test_2`: possible in 3 steps, focus on combination
- `test_3`: possible in 3 steps, focus on conversion
- `test_4`: possible in 3 steps, focus on aggregation
- `test_5`: conversion and aggregation, 
- `test_6`: slightly more complex, possible in 15-23 steps
- `test_7`: possible in 11 steps
- `test_8`: simple context example

Tests in `case_abstract_large`: 
- `test_1`: a large test case, similar to `test_6` from the medium cases, but for larger graphs
- `test_2`: the largest test case

## Choose test case
Adjust code cell below to select one of the test cases mentioned above.

In [4]:
test = test_mobility

# Inspect test case 
Inspect the goal

In [26]:
print("Start set: \n"+ str(test.start_set))
#print("models: "+ str(test.models))
print("Goal: "+ str(test.goal))
print("Models: \n " + "\n ".join([str(x) for x in test.models]))


Start set: 
{(s0 | od0, r0)_['roads_excl_sample', 'roads_in_sample'],
 (s0 | m0, od0)_['roads_excl_sample', 'roads_in_sample'],
 (b0, od0 | p0, t2)_['people_excl_sample', 'people_in_sample'],
 (m0 | p0, t1)_['people_in_sample'],
 (c0 | s0, t0)_['roads_in_sample']
}
Goal: (c0 | s0, t2)_['roads_excl_sample', 'roads_in_sample']
Models: 
 Modality Choice model: (b0, od0 | p0, t2)_['dummyX'] + (m0 | p0, t2)_['dummyY'] -> (p0 | od0, t2)_['dummyY']
 Shortest Path model: (p0 | od0, t2)_['dummyX'] + (s0 | od0, r0)_['dummyY'] -> (p0 | s0, t2)_['dummyY']
 Calibration model: (c0, p0 | s0, t2)_['dummyX'] + (p0 | s0, t2)_['dummyY'] -> (c0 | s0, t2)_['dummyY']


Inspect a specific graph from the test case.

In [7]:
#agg_x = AggregationGraph.get("n")
#print(agg_x.granularities)
#agg_x.plot_graph()

# Run test case (once)

In [25]:
result = a_star(start_set=test.start_set, 
                goal=test.goal,
                models=test.models,
                max_iteration=200, 
                similarity_choice="topsum", # sum
                score_function_parameter=3,  # for topsum: multiplier
                prints=False, 
                preprocess_rhs=True,
                find_multiple_paths=False,
                shedding=True,
                shedding_n=50,
                variant="individual")  # base, normalized_basic, normalized_coupled, individual

# display results
if isinstance(result, SetOfSources):    
    # one path was found
    print("Path: \n   " + "\n   ".join(result.path))
elif isinstance(result, list):
    # multiple paths were found
    print(str(len(result))+" paths were found")
else:
    # no path was found
    print(result)

Path: 
   start_set
   prep_rhs: (m0 | p0, t1)_['people_in_sample'] -> (m0 | p0, t2)_['people_in_sample']
   prep_rhs: (c0 | s0, t0)_['roads_in_sample'] -> (c0 | s0, t2)_['roads_in_sample']
   model (Modality Choice model):(p0 | od0, t2)_['people_excl_sample', 'people_in_sample']
   model (Shortest Path model):(p0 | s0, t2)_['roads_excl_sample', 'roads_in_sample']
   combine (columnwise): (c0, p0 | s0, t2)_['roads_in_sample']
   model (Calibration model):(c0 | s0, t2)_['roads_excl_sample', 'roads_in_sample']


### Find shortest path
If the option find_multiple_paths=True was used, there may now be multiple paths in the result of the A* algorithm. Later, quality assignments will help with choosing a specific path. For now, we are interested in the shortest path.

In [9]:
# Number of paths found
#len(result)

In [10]:
# Determine shortest path 
#path_lengths = [len(x.path) for x in result]
#shortest_path_index = path_lengths.index(min(path_lengths))  # find shortest path index
#shortest_path = result[shortest_path_index]  # take the path with the shortest path
#print("Path: " + "\n      ".join(shortest_path.path))
            

### Run test case (multiple times)

In [11]:
#simulate(n_simulations=10, 
#         start_set=test.start_set,
#         goal=test.goal,
#         models=test.models,
#         max_iteration=50, 
#         similarity_choice="topsum", # sum
#         score_function_parameter=3,  # for topsum: multiplier
#         preprocess_rhs=True,
#         shedding=True,
#         shedding_n=50,
#         variant="individual")

# TODO: add number of iterations and number of times a result was found succesfully